個人課題1　- Github上でGoogleが管理しているリポジトリの情報をスクレイピングして下記の情報を取得する

In [ ]:
import sqlite3
path = ''
db_name = 'github_google.db'

try:
    #DBへのコネクションを確立
    conn = sqlite3.connect(path + db_name)

    #SQL(ROBを操作するための言語)を実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    # SQL文の作成
    # テーブルの作成
    sql = "CREATE TABLE repositories (id INTEGER PRIMARY KEY, name TEXT, pl TEXT, star INTEGER);"

    # SQL文の実行
    cur.execute(sql)

    conn.commit()  # 変更を保存

except sqlite3.Error as e:
    print('SQLite error:', e)
    
finally:
    # DBへの接続を閉じる
    if conn:
        conn.close()

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

repositories = []

page_now = 1

# 95ページ
for page in range(page_now, 96):

    # 長めに待つ
    time.sleep(2)

    url = f"https://github.com/orgs/google/repositories?page={page}"
    print(f"\nページ {page}: {url}")
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    # GitHub 新UIのリポジトリアイテム
    items = soup.find_all(class_="ListItem-module__listItem--k4eMk")
    print(f"ページ{page}のリポジトリ数: {len(items)}")

    if len(items) == 0:
        print("終了")
        break

    for item in items:
        # リポジトリ名
        name_tag = item.find(class_="Title-module__anchor--GmXUE")
        name = name_tag.text.strip() if name_tag else "N/A"

        # 言語
        pl_tag = item.find(class_="ReposListItem-module__Text_4--mkG7R")
        pl = pl_tag.text.strip() if pl_tag else "N/A"

        # スター数
        star = 0
        star_a = item.find("a", href=lambda x: x and x.endswith("/stargazers"))

        if star_a and star_a.has_attr("aria-label"):
            label = star_a["aria-label"]
            num = label.split()[0].replace(",", "")
            if num.endswith("k"):
                star = int(float(num[:-1]) * 1000)
            else:
                star = int(num)

        print(f"name:{name} | pl:{pl} | star:{star}")
        repositories.append((name, pl, star))

print("総件数:", len(repositories))



ページ 1: https://github.com/orgs/google/repositories?page=1
ページ1のリポジトリ数: 30
name:dive | pl:C++ | star:17
name:nearby | pl:C++ | star:888
name:tunix | pl:Python | star:1900
name:clusterfuzz | pl:Python | star:5500
name:osv-scalibr | pl:Go | star:538
name:XNNPACK | pl:C | star:2200
name:xls | pl:C++ | star:1400
name:gvisor | pl:Go | star:17000
name:osv-scanner | pl:Go | star:8100
name:meridian | pl:Python | star:1200
name:closure-templates | pl:Java | star:671
name:perfetto | pl:C++ | star:5000
name:dart_cli_pkg | pl:Dart | star:61
name:adk-python | pl:Python | star:16000
name:nomulus | pl:Java | star:1800
name:boringssl | pl:C++ | star:2100
name:crubit | pl:C++ | star:924
name:orbax | pl:Python | star:455
name:angle | pl:C++ | star:3800
name:android-cuttlefish | pl:C++ | star:571
name:jetpack-camera-app | pl:Kotlin | star:276
name:zerocopy | pl:Rust | star:2100
name:heir | pl:C++ | star:610
name:dawn | pl:C++ | star:780
name:gsocguides | pl:CSS | star:475
name:cameratrapai | pl:Python | 

In [65]:
#DB接続
conn = sqlite3.connect("google_repos.db")
#カーソル取得
cur = conn.cursor()
#SQL定義
cur.execute("""
CREATE TABLE IF NOT EXISTS repositories (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    language TEXT,
    stars INTEGER
)
""")
#SQL実行
cur.executemany(
    "INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?)",
    repositories
)
#コミット
conn.commit()
#接続終了
conn.close()